In [1]:
from general_trader import GeneralTrader
import polars as pl
import polars_talib as pltalib
from trading_helpers import TradingData

In [2]:
traderData = TradingData(
    "/mnt/jonarne/dev/trading_data/",
    "/SP500/",
    "SP500_list.parquet",
)
# download_all_tickers. This function is only needed to call once,
# or when you decide to download new fresh ticker and sector data
#traderData.download_all_tickers()


# populate polars dataframe with open, close, high, low and volume data
ochl_data = traderData.load_ticker_parquet_files()
print(ochl_data.collect())


TradingData Initialized
shape: (4_300_704, 7)
┌─────────────────────┬───────────┬────────────┬───────────┬───────────┬─────────┬────────┐
│ date                ┆ close     ┆ high       ┆ low       ┆ open      ┆ volume  ┆ ticker │
│ ---                 ┆ ---       ┆ ---        ┆ ---       ┆ ---       ┆ ---     ┆ ---    │
│ datetime[ns]        ┆ f64       ┆ f64        ┆ f64       ┆ f64       ┆ i64     ┆ str    │
╞═════════════════════╪═══════════╪════════════╪═══════════╪═══════════╪═════════╪════════╡
│ 1994-10-18 00:00:00 ┆ 1.205047  ┆ 1.214462   ┆ 1.186218  ┆ 1.205047  ┆ 418166  ┆ O      │
│ 1994-10-19 00:00:00 ┆ 1.308607  ┆ 1.318021   ┆ 1.214463  ┆ 1.214463  ┆ 249950  ┆ O      │
│ 1994-10-20 00:00:00 ┆ 1.318019  ┆ 1.365091   ┆ 1.318019  ┆ 1.327434  ┆ 141590  ┆ O      │
│ 1994-10-21 00:00:00 ┆ 1.299191  ┆ 1.327434   ┆ 1.289776  ┆ 1.318019  ┆ 68731   ┆ O      │
│ 1994-10-24 00:00:00 ┆ 1.270948  ┆ 1.327435   ┆ 1.270948  ┆ 1.299192  ┆ 75749   ┆ O      │
│ …                   ┆ …         

In [3]:


    # Define a sample Polars expression
    #my_filter = (pl.col("close") > 100) & (pl.col("volume") > 50000)
my_filter = None
    # --- Test Cases ---

print("--- Testing Valid Instantiation ---")
try:
    trader = GeneralTrader(ohlcColumnsToUse=["close"], df_ohlc=ochl_data.collect(), initialFilter=my_filter)
    print("Successfully created GeneralTrader instance.")
    print("DataFrame Schema:", trader.df_ohlc.schema)
    print("Initial Filter:", trader.initialFilter)
except ValidationError as e:
    print("Validation Error (unexpected):", e)

--- Testing Valid Instantiation ---
Successfully created GeneralTrader instance.
DataFrame Schema: Schema([('close', Float64), ('ticker', String), ('date', Datetime(time_unit='ns', time_zone=None))])
Initial Filter: None


In [4]:
trader.df_ohlc

close,ticker,date
f64,str,datetime[ns]
1.205047,"""O""",1994-10-18 00:00:00
1.308607,"""O""",1994-10-19 00:00:00
1.318019,"""O""",1994-10-20 00:00:00
1.299191,"""O""",1994-10-21 00:00:00
1.270948,"""O""",1994-10-24 00:00:00
…,…,…
94.470001,"""NRG""",2025-04-11 00:00:00
95.919998,"""NRG""",2025-04-14 00:00:00
97.110001,"""NRG""",2025-04-15 00:00:00


In [5]:
indicators = {
    "sma": pltalib.sma(timeperiod=200).over("ticker"),
    "mean_B": pl.col("close").mean().over("ticker"),
    #"bad_stuff": 5,
    "A_plus_B": pl.col("close") + pl.col("close"),
}

filterExpr =  pl.col("A_plus_B") > 100
trader.add_columns_and_prefilter(indicators, filterExpr)

GeneralTrader(df_ohlc=shape: (1_033_762, 6)
┌───────────┬────────┬─────────────────────┬───────────┬───────────┬────────────┐
│ close     ┆ ticker ┆ date                ┆ sma       ┆ mean_B    ┆ A_plus_B   │
│ ---       ┆ ---    ┆ ---                 ┆ ---       ┆ ---       ┆ ---        │
│ f64       ┆ str    ┆ datetime[ns]        ┆ f64       ┆ f64       ┆ f64        │
╞═══════════╪════════╪═════════════════════╪═══════════╪═══════════╪════════════╡
│ 50.253445 ┆ O      ┆ 2019-02-05 00:00:00 ┆ 41.484557 ┆ 22.008229 ┆ 100.506889 │
│ 50.151932 ┆ O      ┆ 2019-02-06 00:00:00 ┆ 41.562488 ┆ 22.008229 ┆ 100.303864 │
│ 50.818882 ┆ O      ┆ 2019-02-07 00:00:00 ┆ 41.645151 ┆ 22.008229 ┆ 101.637764 │
│ 50.804382 ┆ O      ┆ 2019-02-08 00:00:00 ┆ 41.726519 ┆ 22.008229 ┆ 101.608765 │
│ 51.043633 ┆ O      ┆ 2019-02-11 00:00:00 ┆ 41.80863  ┆ 22.008229 ┆ 102.087265 │
│ …         ┆ …      ┆ …                   ┆ …         ┆ …         ┆ …          │
│ 94.470001 ┆ NRG    ┆ 2025-04-11 00:00:00 ┆ 90.291616

In [6]:
trader.df_ohlc

close,ticker,date,sma,mean_B,A_plus_B
f64,str,datetime[ns],f64,f64,f64
50.253445,"""O""",2019-02-05 00:00:00,41.484557,22.008229,100.506889
50.151932,"""O""",2019-02-06 00:00:00,41.562488,22.008229,100.303864
50.818882,"""O""",2019-02-07 00:00:00,41.645151,22.008229,101.637764
50.804382,"""O""",2019-02-08 00:00:00,41.726519,22.008229,101.608765
51.043633,"""O""",2019-02-11 00:00:00,41.80863,22.008229,102.087265
…,…,…,…,…,…
94.470001,"""NRG""",2025-04-11 00:00:00,90.291616,25.908705,188.940002
95.919998,"""NRG""",2025-04-14 00:00:00,90.37925,25.908705,191.839996
97.110001,"""NRG""",2025-04-15 00:00:00,90.477072,25.908705,194.220001
